<a href="https://colab.research.google.com/github/GeoLabUniLaSalle/Python/blob/main/Exercice_webcrawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercice

* Website: https://www.offshore-energy.biz/ ​

* Targeted articles in News section.​

* Crawl 2 pages of news.​

* ​For all these articles, find the following items and print them on the screen:​

 * Date​
 * Article title​
 * Article URL ​
 * Article body​

* Bonus: Save them into an Excel/CSV (if you have some time left)

In [12]:
# Install modules
import sys
!{sys.executable} -m pip install --user bs4
!{sys.executable} -m pip install --user pandas

Restart the kernel after installing the packages

In [3]:
# Import modules
import bs4
import requests
import pandas as pd

# 1. Define starting urls in a list

In [4]:
urls = ['https://www.offshore-energy.biz/news/', 'https://www.offshore-energy.biz/news/page/2/']

# 2. Find article links for every url

In [5]:
links = []

for url in urls:
    # Gets the content of the page 
    html_code = requests.get(url).content
    #print(html_code)

    # Transforms it to a tree of tags
    soup = bs4.BeautifulSoup(html_code, 'html.parser')
    
    # Select article links
    tag_links = soup.select('div.card-rich h3 a') # all tags corresponding are selected

    for link in tag_links:
        links.append(link.get('href')) # selects href from tags and put them into a list

In [6]:
len(links) # verifies how much articles are selected (we expect approx. 50)

51

# 3. Parse articles 
We have a list of article links. \
For every link, we will put the title, body and date of the article in a dictionary \
As we have more than 50 links, it can be long !

In [7]:
parsed_articles = [] # Defines a list to store dictionnary of parsed articles

for art_link in links:
    html_code = requests.get(art_link).content # fetches article content
    soup = bs4.BeautifulSoup(html_code, 'lxml') # transforms raw html into a tree of tags
    
    # Title
    title = soup.select('article h1')[0].get_text() # gets the title. 
                                                    # As get returns a list, we pick the first element [0]
    
    # Body
    body = soup.select('article div.wp-content p') # gets the article body
    body_clean = []
    for paragraph in body:
        body_clean.append(paragraph.get_text()) # appends the text of every paragraph to a list

    body_text = '\n'.join(body_clean) # converts the list of strings into a unique string by joining all strings.
                                    # as '\n' was used to join strings, the space between paragraphs is kept.
    
    # Date
    date = soup.select_one('div.article-meta__info').get_text() # gets date
    date = date.split('by')[0].split('posted')[0].strip() # removes the noise around the date
    # see split and strip() funtions on Stack Overflow. (or see below)
    
    parsed_articles.append({'Title': title,
                           'Date': date,
                           'Body': body_text,
                           'URL': art_link}) # results are is a list of dictionaries

In [8]:
# Example of '\n'.join(list_of_str)
l = ['Hello', 'world']
print(' '.join(l))
print('-----')
print('\n'.join(l))

Hello world
-----
Hello
world


In [9]:
# Example of split
word = 'hello my friend'
split_word = word.split('my')
print(split_word)

['hello ', ' friend']


# 4. Put results in a table (pandas DataFrame)

In [10]:
table = pd.DataFrame(parsed_articles)

In [11]:
table.head(5) # Print first 5 rows

,Title,Date,Body,URL
0,In focus: Converting jack-up rigs and combinin...,"September 24, 2021,",\nThis week saw a plethora of energy transitio...,https://www.offshore-energy.biz/in-focus-conve...
1,Murphy Oil’s giant FPS King’s Quay reaches Texas,"September 22, 2021,",\nMurphy Oil is on schedule to achieve the fir...,https://www.offshore-energy.biz/murphy-oils-gi...
2,Buzzing networking drinks at OEEC!,"September 15, 2021,",Offshore Energy Exhibition and Conference (OEE...,https://www.offshore-energy.biz/buzzing-networ...
3,"Maersk, CCS team up on carbon-neutral technolo...","September 27, 2021,",Danish shipping giant A.P. Moller – Maersk has...,https://www.offshore-energy.biz/maersk-china-c...
4,Odfjell Drilling to bag $20 million from extra...,"September 27, 2021,",Norwegian energy giant Equinor has added an ad...,https://www.offshore-energy.biz/odfjell-drilli...


In [12]:
# More details about an article
article_id = 4
print(table.iloc[article_id])

Title    Odfjell Drilling to bag $20 million from extra...
Date                                   September 27, 2021,
Body     Norwegian energy giant Equinor has added an ad...
URL      https://www.offshore-energy.biz/odfjell-drilli...
Name: 4, dtype: object
